In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
tf.debugging.set_log_device_placement(True)

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.9 MB/s eta 0:00:00


In [ ]:
pip install datasets

In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=61b3f44d66ae4a50cc94cf31623a142a2c01f365d4f0389349a8a1ddd69d5b63
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,
    Seq2SeqTrainer, Seq2SeqTrainingArguments
)
import evaluate
from google.colab import files  # For manual file upload

# 🔹 Step 1: Upload CSV File Manually
uploaded = files.upload()  # Opens a file selection dialog

# Get the uploaded filename
filename = list(uploaded.keys())[0]
print(f"✅ Uploaded File: {filename}")

# Load the dataset
df = pd.read_csv(filename)

# Ensure required columns are present
required_columns = {"Normalized_Text", "Summary"}
missing_columns = required_columns - set(df.columns)
if missing_columns:
    raise ValueError(f"❌ Missing required columns: {missing_columns}")

print(f"✅ Dataset loaded with {len(df)} rows.")

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['Normalized_Text', 'Summary']])

# Split into train/test
dataset = dataset.train_test_split(test_size=0.1)
print(f"✅ Dataset split: {len(dataset['train'])} train | {len(dataset['test'])} test")

# 🔹 Step 2: Load Model & Tokenizer (Choose mBART or mT5)
MODEL_NAME = "facebook/mbart-large-50"  # or "google/mt5-small"
print(f"✅ Loading model: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# 🔹 Step 3: Preprocessing Function
def preprocess_function(examples):
    inputs = tokenizer(
        examples["Normalized_Text"], padding="longest", truncation=True, max_length=512
    )
    targets = tokenizer(
        examples["Summary"], padding="longest", truncation=True, max_length=128
    )
    inputs["labels"] = targets["input_ids"]
    return inputs

# Tokenize dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["Normalized_Text", "Summary"])
print(f"✅ Tokenization completed!")

# 🔹 Step 4: Define Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    logging_steps=500,
    report_to="none",  # Disable reporting (e.g., WandB, TensorBoard)
)

# 🔹 Step 5: Define Trainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v * 100, 2) for k, v in result.items()}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 🔹 Step 6: Train Model
print("🚀 Starting training...")
trainer.train()
print("✅ Training completed!")

# 🔹 Step 7: Save Model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")
print("✅ Fine-tuning completed and model saved to './fine_tuned_model'")

Saving summarized_policies.csv to summarized_policies (1).csv
✅ Uploaded File: summarized_policies (1).csv
✅ Dataset loaded with 500 rows.
✅ Dataset split: 450 train | 50 test
✅ Loading model: facebook/mbart-large-50


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

✅ Tokenization completed!


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-22035557ef01>:87: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


🚀 Starting training...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,0.386797,44.320000,36.610000,41.610000,41.660000
2,No log,0.344612,47.890000,40.870000,45.930000,46.230000
3,No log,0.249030,54.110000,49.180000,52.150000,52.370000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


✅ Training completed!
✅ Fine-tuning completed and model saved to './fine_tuned_model'


In [ ]:
import shutil
import joblib
import pickle
from google.colab import files

# 🔹 Step 7: Save Model
model_dir = "./fine_tuned_model"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

# 🔹 Step 8: Save Using Pickle (Optional)
model_path = "fine_tuned_model.sav"
with open(model_path, "wb") as f:
    pickle.dump(model, f)

print(f"✅ Model saved to {model_path}")

# 🔹 Step 9: Zip and Download Model
shutil.make_archive("fine_tuned_model", 'zip', model_dir)  # Create a ZIP file
files.download("fine_tuned_model.zip")  # Prompt file download
print("✅ Model zipped and ready for download!")

✅ Model saved to fine_tuned_model.sav


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Model zipped and ready for download!
